<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/abstractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.thepythoncode.com/article/text-summarization-using-huggingface-transformers-python
#https://huggingface.co/blog/how-to-generate

# use much smaller sample size. and then re-evaluate extractive models to match so have comparative stats? or just _best_ extractive model!
  # discuss that results are less robust and list as extension to parallelize. 

# probably metrics will be worse because can generate words that mean the same thing but are different - not captured by rouge scores 
  # but does this give us any real value over extractive for our? look at individual instances to get a sense. 

# disadvantages: 
  # max length of input for training (assuming most important info at beginning, could completely miss topics later in article. ok for bcg - pyramid principle)
  # less flexible - cannot specify number of sentences for example 
  # sometimes generates <unk> 
  # can't guarentee will generate full sentences with max settings. can't tell it to stop at last sentence - have to manually trim off last bit of partial sentence. 
# advantages:
  # fewer decision re text cleaning; can consider more features like punctuation and capitalization 

In [ ]:
# TODO create configuration options - length (min? max? penalty?), search type
# TODO evaluate on all summaries in dataframe
# TODO create loop through configurations similar to extractive 
# TODO set up evaluations 

In [1]:
%%capture
!pip install transformers
!pip install import-ipynb
!pip install sentencepiece

In [2]:
from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import torch
import import_ipynb
import numpy as np
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# load in functions from extract_summarization notebook
%cd "drive/MyDrive/Colab Notebooks"
from extractive_summarization import *
%cd ..

/content/drive/MyDrive/Colab Notebooks
importing Jupyter notebook from extractive_summarization.ipynb
/content/drive/My Drive


In [9]:
df = data_setup(n = 20) 

In [10]:
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-small")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [11]:
df['encoded'] = df.sentences.map(lambda row: tokenizer.encode("summarize: " + ' '.join(row), return_tensors="pt", max_length=1017, truncation=True))
# do not use cleaned sentences because want to generate based off of all structures in a sentence - capitalization, punctuation etc. Things like stopwords are important for sentence structure. 

In [26]:
df['max_words'] = df.sentences.map(lambda row: int(np.floor(len(''.join(row).split(' ')) / 20))) # average 20 text words per summary word

In [ ]:
# extension: true grid search through some parameters like length penalty - unable to do due to computation constraints
# also true grid search over number of beams, no repeat ngram number

# extension: sampling methods. Often used to introduce some randomness to the speech and make it sound more human-like. useful for generating stories for ex
  # but in this case don't want randomness. want to stick to the specifics of the inputted text. 
  # ex: article about a man attacked by a tiger says that he was conscious and talking in the ambulance. turns this into - he was "conscious and talking" with the animal

In [27]:
%%time
# greedy (decoding method)
# min length force to give summary around the target (like in extractive)
# no repeat ngram = 4 - want entity bigrams, trigrams to be able to repeat, but not entire phrases 
  # else get repeats like ex: the heat index will make it feel like 113. the heat index will make it feel like 113.
# con: high probability words after a low probability word, so never look at it

df['outputs'] = df.apply(lambda row: model.generate( 
                                     row.encoded, 
                                     max_length=row.max_words, 
                                     min_length=max(0, row.max_words - 20),
                                     no_repeat_ngram_size = 4), 
                         axis = 1) 
# less strict version: can go over limit by extra 20 words (as if 1 more text word)
df['outputs'] = df.apply(lambda row: model.generate( 
                                     row.encoded, 
                                     max_length=row.max_words + 20, 
                                     min_length=max(0, row.max_words - 20),
                                     no_repeat_ngram_size = 4), 
                         axis = 1) 
# don't want min ever 0 such that model decides min. need enough info for content creator to work with. 

CPU times: user 47.9 s, sys: 319 ms, total: 48.3 s
Wall time: 48.2 s


In [54]:
%%time
# beam search - most likely num_beams at each time step. looks at probability over sequence of num_beams long
df['outputs'] = df.apply(lambda row: model.generate( 
                                     row.encoded, 
                                     max_length=row.max_words + 20, 
                                     min_length=max(0, row.max_words - 20),
                                     num_beams = 5,
                                     early_stopping = True),
                         axis = 1) 

CPU times: user 1min 49s, sys: 3.24 s, total: 1min 52s
Wall time: 1min 52s


In [61]:
df['outputs'] = df.apply(lambda row: model.generate( 
                                     row.encoded, 
                                     do_sample = True,
                                     max_length=row.max_words + 20, 
                                     min_length=max(0, row.max_words - 20),
                                     top_k = 50),
                         axis = 1) 

In [ ]:
(10000*90)/60/60 # 250 hours for 10,000  
# 1 min 30 seconds for 10 inputs
# more than douple for 20 inputs = 3 min 30 seconds
# greedy - half the time 

250.0

In [ ]:
CONFIGURATIONS = [['greedy', 'beam'],
                  ['max_words_strict', 'max_words_plus'],
                  ]    

In [64]:
df['predicted_summary'] = df.outputs.map(lambda row: tokenizer.decode(row[0], skip_special_tokens = True))